In [4]:
# import libraries
import pickle
import pandas as pd
import gensim.models.word2vec as w2v
import multiprocessing
import os
import numpy as np

Using TensorFlow backend.


In [2]:
data = pd.read_pickle('../data/quora.pickle')

In [3]:
data.head()

,q1_len,q2_len,q1_voc_len,q2_voc_len,q1_average_word_len,q2_average_word_len,shared_token_len,shared_token_average_word_len,question_1,is_duplicate,question_2
0,67,58,13,12,3.53,3.54,12,3.5,"[what, is, the, step, by, step, guide, to, inv...",0,"[what, is, the, step, by, step, guide, to, inv..."
1,56,93,13,17,3.23,4.11,10,3.3,"[what, is, the, story, of, kohinoor, (, koh, i...",0,"[what, would, happen, if, the, indian, governm..."
2,74,60,15,11,4.00,4.55,5,4.0,"[how, can, i, increase, the, speed, of, my, in...",0,"[how, can, internet, speed, be, increased, by,..."
3,53,68,11,12,3.08,3.27,1,1.0,"[why, am, i, mentally, very, lonely, ?, how, c...",0,"[find, the, remainder, when, math, 23, 24, mat..."
4,81,40,15,8,4.00,4.12,5,3.4,"[which, one, dissolve, in, water, quikly, suga...",0,"[which, fish, would, survive, in, salt, water, ?]"


In [4]:
# remove empty row
ind1 = data[data['q1_len'] == 0].index
ind2 = data[data['q2_len'] == 0].index

In [5]:
ind = ind1.append(ind2)

In [6]:
ind

Int64Index([3306, 190570, 290090, 301583, 20072, 144343, 384293], dtype='int64')

In [7]:
data.drop(ind, inplace=True)

In [8]:
data[['q1_len']].describe()

,q1_len
count,404281.000000
mean,61.163663
std,30.760206
min,1.000000
25%,40.000000
50%,53.000000
75%,74.000000
max,644.000000


In [9]:
data[['q2_len']].describe()

,q2_len
count,404281.000000
mean,61.750359
std,34.777124
min,1.000000
25%,40.000000
50%,52.000000
75%,73.000000
max,1189.000000


In [10]:
q1_greater_than100 = data[data['q1_len']> 100]

In [11]:
q1_greater_than100.shape

(42057, 11)

In [12]:
q1_greater_than100['question_1'].iloc[0]

['what',
 'are',
 'the',
 'laws',
 'to',
 'change',
 'your',
 'status',
 'from',
 'a',
 'student',
 'visa',
 'to',
 'a',
 'green',
 'card',
 'in',
 'the',
 'us',
 ',',
 'how',
 'do',
 'they',
 'compare',
 'to',
 'the',
 'immigration',
 'laws',
 'in',
 'canada',
 '?']

In [13]:
# remove all the index greater than 100
ind100 = data[data['q1_len'] > 100].index
data.drop(ind100, inplace=True)

ind100_2 = data[data['q2_len'] > 100].index
data.drop(ind100_2, inplace=True)

In [14]:
data.shape

(335001, 11)

In [15]:
q1_less_than5 = data[data['q1_len']< 5]

In [16]:
q1_less_than5.shape

(25, 11)

In [17]:
q2_less_than5 = data[data['q2_len']< 5]

In [18]:
q2_less_than5.shape

(9, 11)

In [19]:
# remove all the question less than 5
ind5_1 = data[data['q1_len'] < 5].index
ind5_2 = data[data['q2_len'] < 5].index
ind5 = ind5_1.append(ind5_2)
data.drop(ind5, inplace=True)

In [20]:
data.shape

(334968, 11)

In [21]:
# pad sentences with empty string
#sentences_1 = data['question_1'].apply(lambda x: x + ['']*(100 - len(x)))
#sentences_2 = data['question_2'].apply(lambda x: x + ['']*(100 - len(x)))

In [23]:
sentences_1 = data['question_1']
sentences_2 = data['question_2']

In [24]:
# get all the questions
questions = sentences_1.append(sentences_2, ignore_index=True).tolist()

In [25]:
len(questions)

669936

In [27]:
questions[4]

['which',
 'one',
 'dissolve',
 'in',
 'water',
 'quikly',
 'sugar',
 ',',
 'salt',
 ',',
 'methane',
 'and',
 'carbon',
 'di',
 'oxide',
 '?']

In [3]:

# build model
num_features = 100
min_word_count = 1
num_workers = multiprocessing.cpu_count()
context_size = 5
downsampling = 1e-3
seed = 1
quora2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features, 
    min_count=min_word_count,
    window=context_size,
    sample=downsampling

)

NameError: name 'multiprocessing' is not defined

In [2]:
quora2vec.build_vocab(questions)

NameError: name 'quora2vec' is not defined

In [30]:
# train
quora2vec.train(questions, total_examples=quora2vec.corpus_count, epochs=quora2vec.iter)

23330639

In [31]:
quora2vec

In [32]:
# save the model
quora2vec.save(os.path.join("../data","quora.w2v"))

In [38]:
def get_vector(sentence, model, size=300):
    '''get a sentence and return its word2vec representation matrix'''
    sentence_len = len(sentence)
    sentence2vec = np.zeros(shape=(sentence_len, size), dtype='float32')
    for i in range(sentence_len):
        word = sentence[i]
        w = model[word]
        sentence2vec[i] = w
    return np.mean(sentence2vec, axis=0)

In [34]:
d1 = sentences_1[:5]

In [35]:
len(d1)

5

In [39]:
d1vec = d1.apply(lambda x: get_vector(x, quora2vec))

In [40]:
print('sentence: ', sentences_1[0])
print('vector: ', d1vec[0])

sentence:  ['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india', '?']
vector:  [ -1.76989492e-02  -1.93056375e-01  -2.69635040e-02   2.68157661e-01
  -2.79676653e-02   2.62810886e-01  -2.85850093e-02   1.83542464e-02
   5.97551838e-02   2.00882435e-01   5.61818369e-02   1.76033497e-01
  -8.62299427e-02  -5.75742088e-02  -4.32833694e-02   2.38263339e-01
  -2.17663765e-01   3.48790293e-03   9.32369158e-02  -4.79277149e-02
  -1.95169702e-01  -4.71515030e-01   8.96158069e-02   6.34968374e-03
  -2.65770584e-01  -5.89184724e-02   6.29672259e-02  -4.20912281e-02
   1.89381056e-02  -1.07566483e-01   3.63601685e-01  -1.28117979e-01
   2.29109917e-02   1.09679691e-01   8.12438130e-03   1.66804299e-01
   1.46831051e-01  -1.53821209e-04   1.97601721e-01  -1.90462932e-01
   6.57959953e-02  -1.23254187e-01  -2.08822384e-01   8.22132006e-02
  -9.18527842e-02  -3.34942527e-02   2.07810819e-01  -2.38584392e-02
   1.54657498e-01   1.54791430e-01  -3

In [41]:
len(d1vec[0])

300

In [42]:
question_1 = sentences_1.apply(lambda x: get_vector(x, quora2vec))

In [43]:
question_2 = sentences_2.apply(lambda x: get_vector(x, quora2vec))

In [44]:
# create a word2vec data for quora dataset
### 
is_dup = data['is_duplicate']
quora_df = pd.DataFrame({'question_1': question_1, 'question_2': question_2, 'is_duplicate': is_dup})

In [46]:
quora_df.head(2)

,is_duplicate,question_1,question_2
0,0,"[-0.0176989, -0.193056, -0.0269635, 0.268158, ...","[-0.0454762, -0.207512, -0.0136385, 0.260932, ..."
1,0,"[0.00829708, -0.0273363, 0.0410704, 0.133371, ...","[0.0121135, -0.155123, -0.0672922, 0.198632, -..."


In [ ]:
quora_df.head()

In [47]:
quora_df.shape

(334968, 3)

In [48]:
data.shape

(334968, 11)

In [51]:
quora_df.to_pickle(os.path.join('../data/', 'quoraw2v.pickle'))